# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [8]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [9]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [ ]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [10]:
from dotenv import load_dotenv, find_dotenv

find_dotenv()

''

In [11]:
load_dotenv(find_dotenv())

False

In [12]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"
os.environ["SERPER_API_KEY"] = ""

![alt text](<Screenshot (237).png>)

![alt text](<Screenshot (236).png>)

* Verstile means it can handle different types of inputs
* Fault-tolerant means if an exception/error occurs so it doesn't stop execution.Instead of that it's fail gracefully(and try again) and ask agent tp retry given the error message
* Caching means not doing a request unnecessary 

![alt text](<Screenshot (238).png>)


![alt text](<Screenshot (239).png>)



![alt text](<Screenshot (240).png>)

## Creating Agents

In [13]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [14]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [15]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [17]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [18]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [19]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [20]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [21]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [22]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [23]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=True,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [24]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [2024-09-01 22:14:07][DEBUG]: == Working Agent: Sales Representative
 [2024-09-01 22:14:07][INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...


Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.40it/s]


I need to gather data on DeepLearningAI, focusing on their company background, key personnel, recent business developments, and potential needs that align with our offerings. To start this process, I will search the internet for relevant information about DeepLearningAI.

Action: Search the internet  
Action Input: {"search_query": "DeepLearningAI company profile key decision makers recent developments"}  
 


Search results: Title: DEEPLEARNING.AI LLC Company Profile | Palo Alto, CA
Link: https://www.dnb.com/business-directory/company-profiles.deeplearningai_llc.7cb525da4ad45b067062d87d5fbcad0b.html
Snippet: Who is DEEPLEARNING.AI LLC's key principal? DEEPLEARNING.AI LLC's key principal is Andrew Ng. What year was DEEPLEARNING.AI LLC started?
---
Title: DeepLearning.AI - LinkedIn
Link: https://www.linkedin.com/company/deeplearningai
Snippet: DeepLearning.AI is making a world-class AI education accessible to people around the globe. DeepLearning.AI was founded by Andrew Ng, a global le

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.53it/s]


Thought: I found some relevant links regarding DeepLearningAI, specifically their company profile and leadership. I will start by reading the profile from the first search result to gather foundational information about the company.

Action: Read a file's content  
Action Input: {"file_path": "https://www.dnb.com/business-directory/company-profiles.deeplearningai_llc.7cb525da4ad45b067062d87d5fbcad0b.html"}  
 

Fail to read the file https://www.dnb.com/business-directory/company-profiles.deeplearningai_llc.7cb525da4ad45b067062d87d5fbcad0b.html. Error: [Errno 22] Invalid argument: 'https://www.dnb.com/business-directory/company-profiles.deeplearningai_llc.7cb525da4ad45b067062d87d5fbcad0b.html'



...

Thought: I will now finalize the email drafts tailored for Andrew Ng and ensure they are compelling and aligned with DeepLearningAI's corporate identity.  
Action: Sentiment Analysis Tool  
Action Input: {"text": "Hello Andrew,\n\nAs a leader in AI education, your dedication to providing world-class learning opportunities stands out. Our solutions can help organizations like yours enhance efficiency and effectiveness in delivering educational content.\n\nLet's discuss how we can contribute to your success story.\n\nWarm regards,\n[Your Name]"}   

positive



Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.39it/s]


Thought: I need to ensure that the final email drafts resonate positively and are tailored to Andrew Ng's leadership and DeepLearningAI's mission.  
Action: Sentiment Analysis Tool  
Action Input: {"text": "Dear Andrew,\n\nCongratulations on the innovative work at DeepLearningAI! Our solutions are designed for tech startups, offering flexibility and scalability to support your growth.\n\nLet's explore how we can work together to enhance your educational offerings.\n\nBest,\n[Your Name]"}   

I tried reusing the same input, I must stop using this action input. I'll try something else instead.





Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.53it/s]


Thought: I should finalize the email drafts tailored for Andrew Ng, ensuring they resonate with his leadership and DeepLearningAI's mission.  
Action: Search the internet  
Action Input: {"search_query": "Andrew Ng DeepLearningAI recent product launch"}   


Search results: Title: DeepLearning.AI: Start or Advance Your Career in AI
Link: https://www.deeplearning.ai/
Snippet: Join over 7 million people learning how to use and build AI. AI Courses and Specializations. Build a foundation of machine learning and AI skills.
---
Title: Machine Learning Specialization - DeepLearning.AI
Link: https://www.deeplearning.ai/courses/machine-learning-specialization/
Snippet: New Machine Learning Specialization, an updated foundational program for beginners created by Andrew Ng | Start Your AI Career Today.
---
Title: The Batch | DeepLearning.AI | AI News & Insights
Link: https://www.deeplearning.ai/the-batch/
Snippet: The Batch AI News and Insights: In the last couple of days, Google announced a dou

- Display the final result as Markdown.

In [25]:
from IPython.display import Markdown
Markdown(result.raw)

**Email Draft 1: Focused on Strategic Partnership**  
```
Subject: Collaboration Opportunity to Enhance AI Education

Dear Andrew,

I hope this message finds you well. I wanted to extend my congratulations on the recent launch of "Generative AI for Everyone." It’s inspiring to see DeepLearningAI pushing the boundaries of AI education and making complex concepts accessible to a broader audience.

As you continue to expand your educational offerings, we believe that a strategic partnership could significantly enhance the learner experience. Our solutions are designed to integrate seamlessly with existing platforms, providing advanced AI technologies that can bring practical applications into your curriculum.

Let’s discuss how we can work together to support DeepLearningAI’s mission of empowering individuals and organizations through AI education.

Best regards,  
[Your Name]  
```

**Email Draft 2: Highlighting Efficiency in Educational Content Delivery**  
```
Subject: Enhancing Efficiency in AI Education Delivery

Hello Andrew,

Congratulations on the successful launch of "Generative AI for Everyone"! Your dedication to democratizing AI knowledge truly sets DeepLearningAI apart in the educational landscape.

To further bolster your efforts, our solutions can help streamline the delivery of educational content, ensuring that your learners receive the most effective and engaging experience. We understand the unique challenges of delivering world-class education and believe we can help enhance efficiency without compromising quality.

I would love to discuss how we can contribute to DeepLearningAI’s success story and help shape the future of AI education.

Warm regards,  
[Your Name]  
```

**Email Draft 3: Emphasizing Innovation and Scalability**  
```
Subject: Supporting Your Innovative Journey at DeepLearningAI

Dear Andrew,

Congratulations on the innovative strides DeepLearningAI is making with the "Generative AI for Everyone" course! Your commitment to fostering understanding of advanced AI concepts is commendable.

As you look to scale your educational offerings further, our solutions are tailored for flexibility and scalability, designed to support organizations like yours in delivering cutting-edge courses. Together, we can enhance the educational journey for learners and keep your curriculum aligned with the latest industry trends.

I’m excited about the potential for collaboration and would love to explore how we can work together to elevate DeepLearningAI’s educational impact.

Best,  
[Your Name]  
```

These drafts are tailored to resonate with Andrew Ng's leadership and DeepLearningAI's mission, while also addressing their recent product launch and how our solutions can support their goals.